### ConfigParser Example

In [1]:
from slurmflow.config import ConfigParser
from pprint import pprint as print
import argparse


The `config` module of `slurmflow` contains the `ConfigParser` class, which extends the YAML file format to support string substitution.

In [2]:
cfg = ConfigParser("config/example.yml")

YAML files store key-value pairs in a hierarchical format using headers. The structure of a YAML file is equivalent to a nested dict.

In [3]:
print(cfg.config_data)

{'params': {'batch_size': 8, 'dim': 64, 'learning_rate': 0.001},
 'paths': {'data': '{{paths.mnt}}/data',
           'mnt': '/path/to/mount/dir',
           'models': '{{paths.data}}/bs={{params.batch_size}}_lr={{params.learning_rate}}_dim={{params.dim}}'}}


Notice that some values contain double curly braces `{{ }}`. The `ConfigParser` class introduces `{{ }}` as a new syntactical element to the YAML file format. Paths to other entries of the YAML file can be referenced within the curly braces as `path.to.key` (for example `{{paths.data}}` in the cell below).

In [4]:
cfg.config_data['paths']['models']

'{{paths.data}}/bs={{params.batch_size}}_lr={{params.learning_rate}}_dim={{params.dim}}'

The string substitutions are evaluated across the entire YAML file by the `compile` method of the `ConfigParser`.

In [5]:
cfg.compile()

{'params.batch_size': 8,
 'params.learning_rate': 0.001,
 'params.dim': 64,
 'paths.mnt': '/path/to/mount/dir',
 'paths.data': '/path/to/mount/dir/data',
 'paths.models': '/path/to/mount/dir/data/bs=8_lr=0.001_dim=64'}

The `as_args` flag returns the compiled contents in the form of a `Namespace` &agrave; la `argparse`

In [6]:
args = cfg.compile(as_args=True)
print(f"{args.params.dim=}")
print(f"{args.paths.models=}")

'args.params.dim=64'
"args.paths.models='/path/to/mount/dir/data/bs=8_lr=0.001_dim=64'"


The `leaves` flag compiles the config file without including information about headers in the output. This is useful for integration with `argparse`, since `argparse` namespaces are not hierarchical.

In [7]:
args = cfg.compile(leaves=True, as_args=True)
print(f"{args.dim=}")
print(f"{args.models=}")

'args.dim=64'
"args.models='/path/to/mount/dir/data/bs=8_lr=0.001_dim=64'"


In [14]:
# Initialize an example Namespace
namespace = argparse.Namespace(dim=32, batch=8)
print(namespace)

Namespace(dim=32, batch=8)


In [18]:
# Override the Namespace entries with the contents of the config
namespace.__dict__.update(cfg.compile(leaves=True))
print(namespace)

Namespace(dim=64, batch=8, batch_size=32, learning_rate=0.001, mnt='/path/to/mount/dir', data='/path/to/mount/dir/data', models='/path/to/mount/dir/data/bs=32_lr=0.001_dim=64')


Or the string substitutions can be evaluated for a single entry using the `get` method:

In [10]:
cfg.get('paths.models')

'/path/to/mount/dir/data/bs=8_lr=0.001_dim=64'

Entries of the `ConfigParser` can be modified in-memory using the `set` method. Note that `compile` must be called for the changes to take effect.

In [11]:
cfg.set('params.batch_size', 32)
cfg.compile()

{'params.batch_size': 32,
 'params.learning_rate': 0.001,
 'params.dim': 64,
 'paths.mnt': '/path/to/mount/dir',
 'paths.data': '/path/to/mount/dir/data',
 'paths.models': '/path/to/mount/dir/data/bs=32_lr=0.001_dim=64'}